In [2]:
from stsci.tools import capable
capable.OF_GRAPHICS = False

from pyraf import iraf
from pyraf.iraf import gemini, gemtools, gmos, onedspec
import fileSelect as fs
import copy, os
from astropy.io import fits


In [10]:
datadirall = '../raw/'
databasename = datadirall+'obsLog.sqlite3'
dbFile= datadirall+'obsLog.sqlite3'

qd = {'Full':{'use_me':1,
       'Instrument':'GMOS-N','CcdBin':'1 4','RoI':'Full',
       'Disperser':'R400+_G5305','CentWave':656.0,'AperMask':'1.5arcsec',
       'Object':'1WGA J1011.9+5424',
       'DateObs':'2018-09-15:2020-09-30'}
      }


gmos.gstransform.unlearn()
transFlags = {
    'fl_vardq':'yes','interptype':'linear','fl_flux':'yes',
    'logfile':'gstransLog.txt'
}
# The sky regions should be selected with care, using e.g. prows/pcols:
#   pcols ("tAM2306b.fits[SCI]", 1100, 2040, wy1=40, wy2=320)
gmos.gsskysub.unlearn()
skyFlags = {
    'fl_oversize':'no','fl_vardq':'yes','logfile':'gsskysubLog.txt'
}
#sciFiles = fs.fileListQuery(dbFile, fs.createQuery('sciSpec', qs), qs)


In [24]:
arcname2 = 'gsN20191127S0396'

In [25]:

# Science exposures
r = 'Full'
sciFiles = fs.fileListQuery(dbFile, fs.createQuery('sciSpec', qd[r]), qd[r])
sciFiles

[u'N20191127S0262',
 u'N20191127S0263',
 u'N20191127S0264',
 u'N20191127S0265',
 u'N20191127S0266',
 u'N20191127S0267',
 u'N20191203S0187',
 u'N20191203S0188',
 u'N20191205S0097',
 u'N20191205S0098',
 u'N20191205S0099',
 u'N20191205S0100',
 u'N20191205S0101',
 u'N20191205S0102',
 u'N20191205S0105',
 u'N20191205S0107',
 u'N20191205S0108',
 u'N20191205S0109',
 u'N20191205S0110']

In [26]:
iraf.task(lacos_spec='lacos_spec.cl')
iraf.stsdas

<IrafPkg stsdas (stsdas$stsdas.cl) Pkg: clpackage Bin: bin$>

In [36]:
for sci in sciFiles:
    namesci = 'gs'+sci

    gmos.gstransform (namesci, wavtraname=arcname2, **transFlags)
    nameouttransform = 'tgs'+sci
    gmos.gsskysub (nameouttransform, long_sample=sky, **skyFlags)
    nameoutsky = 'stgs'+sci

    namesout = 'xstgs'+sci
    iraf.gemcrspec(nameoutsky, namesout, logfile='crrej.log', \
              key_gain='GAIN', key_ron='RDNOISE', xorder=9, \
              yorder=-1, sigclip=4.5, sigfrac=0.5, objlim=1., \
              niter=4, verbose='yes', fl_vardq='yes')

-------------------------------------------------------------------------------
GSTRANSFORM -- Thu Jan 30 15:51:06 CST 2020
 
ERROR - GSTRANSFORM: Output image tgsN20191127S0262 exists
ERROR - GSTRANSFORM: 1 errors found with input parameters. Exiting.

GSTRANSFORM exit status: error
-------------------------------------------------------------------------------

----------------------------------------------------------------------------
GSSKYSUB -- Thu Jan 30 15:51:06 CST 2020

input        = tgsN20191127S0262
output       = 
outpref      = s
fl_vardq     = yes
fl_oversize  = no
sci_ext      = SCI
var_ext      = VAR
dq_ext       = DQ
long_sample  = 100-200
mos_sample   = 0.9
mosobjsize   = 1.0
naverage     = 1
function     = chebyshev
order        = 1
low_reject   = 2.5
high_reject  = 2.5
niterate     = 2
grow         = 0.0
fl_inter     = no

ERROR - GSSKYSUB: Output image stgsN20191127S0262 exist
ERROR - GSSKYSUB: 1 error(s) found. Exiting.

GSSKYSUB done. Exit status bad. -- Thu Ja

In [33]:
for sci in sciFiles[0:1]:
    namesout = 'xtgs'+sci
    print(sci)

N20191127S0262


In [35]:
sky = '100-200'
for sci in sciFiles[0:1]:
    namesout = 'tgs'+sci



----------------------------------------------------------------------------
GSSKYSUB -- Thu Jan 30 15:49:52 CST 2020

input        = tgsN20191127S0262
output       = 
outpref      = s
fl_vardq     = yes
fl_oversize  = no
sci_ext      = SCI
var_ext      = VAR
dq_ext       = DQ
long_sample  = 100-200
mos_sample   = 0.9
mosobjsize   = 1.0
naverage     = 1
function     = chebyshev
order        = 1
low_reject   = 2.5
high_reject  = 2.5
niterate     = 2
grow         = 0.0
fl_inter     = no

 
GSSKYSUB: Sky-subtraction started
 
  Input image     = tgsN20191127S0262.fits
  Output image    = stgsN20191127S0262.fits
  Observing mode  = LONGSLIT
  Number of slits = 1
  Slit #1 ; background sample = [100-200]
         RMS of the fit (for VAR plane propagation) = 417.2498

GSSKYSUB done. Exit status good -- Thu Jan 30 15:50:04 CST 2020
-------------------------------------------------------------------------------

